In [2]:
import os
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
import mne

# =========================
# CONFIG
# =========================
BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
META_FILE = os.path.join(BASE_DIR, "participants.csv")

FRONTAL = ['FZ', 'F1', 'F2', 'F3', 'F4', 'FP1', 'FP2']
CENTRAL = ['CZ', 'C1', 'C2', 'C3', 'C4']

# =========================
# LOAD METADATA (PD only)
# =========================
meta = pd.read_csv(META_FILE, dtype={"participant_id": str})
meta = meta[meta["GROUP"].str.upper() == "PD"]

moca_col = "MOCA" if "MOCA" in meta.columns else "MoCA"

# =========================
# HELPERS
# =========================
def normalize_pli_windows(pli):
    """
    Ensure PLI windows have shape (windows, channels, channels)
    Handles common shapes like (60,60,1) or (1,60,60)
    """
    pli = np.asarray(pli)
    pli = np.squeeze(pli)

    if pli.ndim == 2:
        pli = pli[None, :, :]  # add window dim

    if pli.ndim != 3 or pli.shape[1] != pli.shape[2]:
        raise ValueError(f"PLI shape invalid after normalization: {pli.shape}")

    return pli

def compute_frontal_central_plv(pli_windows, ch_names):
    """
    Mean PLV between frontal and central electrodes across all windows
    """
    pli_windows = normalize_pli_windows(pli_windows)
    ch_names = [c.upper() for c in ch_names]

    f_idx = [ch_names.index(c) for c in FRONTAL if c in ch_names]
    c_idx = [ch_names.index(c) for c in CENTRAL if c in ch_names]

    if len(f_idx) == 0 or len(c_idx) == 0:
        return np.nan

    vals = []
    for w in range(pli_windows.shape[0]):
        mat = pli_windows[w]
        vals.extend(mat[np.ix_(f_idx, c_idx)].ravel())

    return np.nanmean(vals)

# =========================
# MAIN LOOP
# =========================
rows = []

for _, row in meta.iterrows():
    subj = row["participant_id"]
    subj_dir = os.path.join(BASE_DIR, subj, "data")

    # ---- Load microstate features
    ms_file = os.path.join(subj_dir, f"{subj}_microstate_global.csv")
    if not os.path.exists(ms_file):
        continue

    ms_df = pd.read_csv(ms_file)
    ms_theta = ms_df[ms_df["band"] == "theta"]
    if ms_theta.empty:
        continue

    # MS-D = class D = MS4
    msd_occ = ms_theta["MS4_occurrence"].values[0]

    # ---- Load theta PLV windows
    pli_file = os.path.join(subj_dir, f"{subj}_theta_pli_windows.npy")
    epo_file = os.path.join(BASE_DIR, subj, f"epo_{subj[-3:]}_raw.fif")
    if not os.path.exists(pli_file) or not os.path.exists(epo_file):
        continue

    pli_windows = normalize_pli_windows(np.load(pli_file))
    epochs = mne.read_epochs(epo_file, preload=False, verbose=False)
    ch_names = epochs.ch_names

    try:
        fc_plv = compute_frontal_central_plv(pli_windows, ch_names)
    except ValueError as e:
        print(f"{subj}: skipped due to PLI shape error -> {e}")
        continue

    rows.append({
        "subject": subj,
        "MSD_theta_occurrence": msd_occ,
        "FC_theta_PLV": fc_plv,
        "MoCA": row[moca_col]
    })

# =========================
# ANALYSIS
# =========================
df = pd.DataFrame(rows).dropna()

# 1. Cross-feature coupling
rho_cf, p_cf = spearmanr(df["MSD_theta_occurrence"], df["FC_theta_PLV"])

# 2. Cognitive relevance of coupling
coupling = df["MSD_theta_occurrence"] * df["FC_theta_PLV"]
rho_moca, p_moca = spearmanr(coupling, df["MoCA"])

print("\n=== CROSS-FEATURE COUPLING (PD only) ===")
print(f"MS-D occurrence × FC theta PLV: rho={rho_cf:.3f}, p={p_cf:.4g}")
print(f"(MS-D × PLV) vs MoCA: rho={rho_moca:.3f}, p={p_moca:.4g}")


C:\Users\User\AppData\Local\Temp\ipykernel_31484\1291002494.py:92: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-001\epo_001_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epo_file, preload=False, verbose=False)
C:\Users\User\AppData\Local\Temp\ipykernel_31484\1291002494.py:92: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-002\epo_002_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epo_file, preload=False, verbose=False)
C:\Users\User\AppData\Local\Temp\ipykernel_31484\1291002494.py:92: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-003\epo_003_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  


=== CROSS-FEATURE COUPLING (PD only) ===
MS-D occurrence × FC theta PLV: rho=0.059, p=0.7741
(MS-D × PLV) vs MoCA: rho=0.053, p=0.7985


C:\Users\User\AppData\Local\Temp\ipykernel_31484\1291002494.py:92: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-025\epo_025_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epo_file, preload=False, verbose=False)
C:\Users\User\AppData\Local\Temp\ipykernel_31484\1291002494.py:92: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-026\epo_026_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epo_file, preload=False, verbose=False)
